# Detección robusta de robo de hidrocarburos con Autoencoder LSTM

Este cuaderno implementa un pipeline reproducible para entrenar, calibrar y evaluar un autoencoder LSTM sobre datos de operación de bombas de rebombeo. El enfoque está optimizado para detectar comportamientos anómalos asociados al robo de hidrocarburos.

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             precision_score, recall_score,
                             roc_auc_score, average_precision_score)
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración global reproducible
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:

# Cargar y ordenar los datos
DATA_PATH = Path('../data/rebombeo_huachicoleo.csv')
df = pd.read_csv(DATA_PATH, parse_dates=['timestamp']).sort_values('timestamp').reset_index(drop=True)

print(f"Total de registros: {len(df):,}")
df.head()


In [ ]:

# Definir hiperparámetros y utilidades de generación de ventanas
WINDOW = 30  # minutos por ventana
STEP = 5     # salto entre ventanas
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
FEATURES = ["flow", "pressure", "pump_rpm", "tank_level", "power"]


def generate_windows(values: np.ndarray, labels: np.ndarray, timestamps: pd.Series,
                     window: int, step: int):
    """Genera ventanas deslizantes, etiquetas agregadas y el timestamp final de cada ventana."""
    X, y, ts = [], [], []
    for start in range(0, len(values) - window + 1, step):
        end = start + window
        X.append(values[start:end])
        y.append(int(labels[start:end].max()))
        ts.append(timestamps.iloc[end - 1])
    return np.array(X), np.array(y), pd.Series(ts, name='window_end')


def build_split(start_idx: int, end_idx: int):
    """Construye ventanas dentro del intervalo [start_idx, end_idx)."""
    values = scaled_values[start_idx:end_idx]
    labels = label_array[start_idx:end_idx]
    timestamps = df.loc[start_idx:end_idx - 1, 'timestamp']
    return generate_windows(values, labels, timestamps, WINDOW, STEP)


In [ ]:

# Dividir los datos por tiempo y escalar usando únicamente registros normales de entrenamiento
n_samples = len(df)
train_end = int(n_samples * TRAIN_RATIO)
val_end = int(n_samples * (TRAIN_RATIO + VAL_RATIO))

df_train = df.iloc[:train_end]
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

label_array = df['label'].values

scaler = StandardScaler()
train_normal_mask = (df.index < train_end) & (df['label'] == 0)
scaler.fit(df.loc[train_normal_mask, FEATURES])
scaled_values = scaler.transform(df[FEATURES])

print(f"Registros entrenamiento: {len(df_train):,} | validación: {len(df_val):,} | prueba: {len(df_test):,}")


In [ ]:

# Crear ventanas para cada partición temporal
X_train_all, y_train_all, ts_train = build_split(0, train_end)
X_val, y_val, ts_val = build_split(train_end, val_end)
X_test, y_test, ts_test = build_split(val_end, n_samples)

# Solo ventanas normales para entrenamiento del autoencoder
X_train = X_train_all[y_train_all == 0]

print(f"Ventanas de entrenamiento: {len(X_train):,} (solo normales)")
print(f"Ventanas de validación: {len(X_val):,} | Ventanas de prueba: {len(X_test):,}")


In [ ]:

# Definir arquitectura del Autoencoder LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

TIMESTEPS = X_train.shape[1]
N_FEATURES = X_train.shape[2]

inputs = Input(shape=(TIMESTEPS, N_FEATURES))
encoded = LSTM(64, activation='tanh', return_sequences=True, dropout=0.1)(inputs)
encoded = LSTM(32, activation='tanh', dropout=0.1)(encoded)

latent = RepeatVector(TIMESTEPS)(encoded)

decoded = LSTM(32, activation='tanh', return_sequences=True, dropout=0.1)(latent)
decoded = LSTM(64, activation='tanh', return_sequences=True, dropout=0.1)(decoded)
outputs = TimeDistributed(Dense(N_FEATURES))(decoded)

autoencoder = Model(inputs, outputs, name='lstm_autoencoder')
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
autoencoder.summary()


In [ ]:

# Entrenar el modelo con parada temprana basada en validación reconstructiva
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = autoencoder.fit(
    X_train, X_train,
    epochs=100,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=1
)

plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.xlabel('Época')
plt.ylabel('MSE')
plt.title('Curvas de entrenamiento del autoencoder')
plt.legend()
plt.show()


In [ ]:

# Funciones auxiliares para evaluación

def reconstruction_errors(model, X):
    reconstructions = model.predict(X, verbose=0)
    return np.mean(np.square(X - reconstructions), axis=(1, 2))


def calibrate_threshold(errors_normals: np.ndarray, errors_val: np.ndarray, labels_val: np.ndarray):
    candidate_percentiles = np.linspace(80, 99, 40)
    best_threshold, best_f1 = None, -np.inf
    metrics = []
    for p in candidate_percentiles:
        threshold = np.percentile(errors_normals, p)
        preds = (errors_val > threshold).astype(int)
        f1 = f1_score(labels_val, preds, zero_division=0)
        precision = precision_score(labels_val, preds, zero_division=0)
        recall = recall_score(labels_val, preds, zero_division=0)
        metrics.append({'percentil': p, 'threshold': threshold, 'precision': precision, 'recall': recall, 'f1': f1})
        if f1 > best_f1:
            best_threshold, best_f1 = threshold, f1
    return best_threshold, pd.DataFrame(metrics)


In [ ]:

# Calibrar umbral con el conjunto de validación
val_errors = reconstruction_errors(autoencoder, X_val)
val_normal_errors = val_errors[y_val == 0]

threshold, calibration_df = calibrate_threshold(val_normal_errors, val_errors, y_val)

print(f"Umbral óptimo (max F1 en validación): {threshold:.6f}")
calibration_df.sort_values('f1', ascending=False).head()


In [ ]:

# Evaluar desempeño en el conjunto de prueba
train_errors = reconstruction_errors(autoencoder, X_train)
test_errors = reconstruction_errors(autoencoder, X_test)

y_pred_test = (test_errors > threshold).astype(int)

report = classification_report(y_test, y_pred_test, target_names=['Normal', 'Anomalía'], zero_division=0, output_dict=True)
report_df = pd.DataFrame(report).transpose()

cm = confusion_matrix(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, test_errors)
avg_precision = average_precision_score(y_test, test_errors)

print("Métricas de clasificación (prueba):")
display(report_df)
print(f"ROC AUC (errores): {roc_auc:.3f}")
print(f"Average Precision: {avg_precision:.3f}")

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicción Normal', 'Predicción Anomalía'],
            yticklabels=['Real Normal', 'Real Anomalía'])
plt.title('Matriz de confusión - prueba')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()


In [ ]:

# Visualizaciones complementarias

fig, ax = plt.subplots(1, 1, figsize=(10, 4))
ax.plot(ts_test, test_errors, label='Error reconstrucción')
ax.axhline(threshold, color='red', linestyle='--', label='Umbral calibrado')
ax.set_xlabel('Timestamp fin de ventana')
ax.set_ylabel('MSE')
ax.set_title('Errores de reconstrucción en prueba')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.kdeplot(train_errors, label='Entrenamiento (normal)')
sns.kdeplot(test_errors[y_test == 0], label='Prueba - normales')
sns.kdeplot(test_errors[y_test == 1], label='Prueba - anomalías')
plt.axvline(threshold, color='red', linestyle='--', label='Umbral')
plt.xlabel('Error de reconstrucción')
plt.ylabel('Densidad')
plt.title('Distribución de errores por conjunto')
plt.legend()
plt.show()


## Resumen de acciones recomendadas

- **Pipeline reproducible:** separación temporal y escalado basados únicamente en datos normales de entrenamiento.
- **Calibración basada en validación:** el umbral se selecciona maximizando F1 y equilibrando precisión/recall.
- **Métricas auditables:** se reportan F1, precisión, recall, ROC-AUC y Average Precision en el conjunto de prueba, junto con matrices de confusión y distribuciones de error.

Este flujo deja listos los componentes para empaquetar el modelo, escalarlo a producción y generar reportes periódicos de desempeño.